In [48]:
import json
import sys
import math
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LinearRegression
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import jieba
import pickle
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/feiluhuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/feiluhuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
review_path = 'train_data/before_comment.csv'
review_df = pd.read_csv(review_path)
review_df = review_df.iloc[:,1:]
review_df = review_df.dropna(axis=0,how='any')
review_df

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,Smoking,Park
0,xXgE_rj-tK9v_MIH3mXpaQ,3W-lOOaIVQZONZP83kab6g,Reggie and company cook up the real deal at th...,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8CSdj8LF593qwMcduzHglQ,3W-lOOaIVQZONZP83kab6g,Convential Wisdom is that this is the best res...,5.0,3.5,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
2,JjXEPAR_tf3VCdngVS9GMg,3W-lOOaIVQZONZP83kab6g,Finally an organic grocery store in East Nashv...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,cBHzDxKCsosdDI4UR3w-eg,3W-lOOaIVQZONZP83kab6g,"This converted laundromat is a small, trendy s...",3.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,tU692E8N0xBQ7Ogc78gN2g,3W-lOOaIVQZONZP83kab6g,This is a great place to meet all sorts of peo...,3.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588250,wbaeORfUCcxkwdh7mNapRg,8j1OffLqKtetcAG3xv3f0Q,We had such a good time! Scott Hinds was very ...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
588251,cJGdlK7KeE78Qa0FydikIw,VTfl9PALOCiGV8SUBpCZEQ,We had a great buying experience. Michael Harr...,5.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
588252,U7esGceQIZ_1IG6Ib0c1lQ,m0-9woIKbK-R4F8RVEe4BQ,My colleague ordered on my behalf and she came...,5.0,2.5,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
588253,Y4uWSRQ-GyOaxA2c-qZxWw,QTVdM8ThpshrX-8GTcdk2w,Love this place! Great for toys and maternity...,5.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
user_groups = dict(list(review_df.groupby('user_id')))

In [4]:
ct = 0 
selected = {}
len_dict = {}
for user, df in tqdm(user_groups.items()):
    if ct < 100:
        selected[user] = df
        len_dict[user] = df.shape[0]
    else:
        if df.shape[0] > min(len_dict.values()):
            k = list(len_dict.keys())
            v = list(len_dict.values())
            selected.pop(k[v.index(min(len_dict.values()))])
            len_dict.pop(k[v.index(min(len_dict.values()))])
            selected[user] = df
            len_dict[user] = df.shape[0]
    ct += 1
len_dict 

100%|██████████| 215518/215518 [00:01<00:00, 199976.24it/s]


{'-SigNCNItQDtSq27j0t_tw': 170,
 '0-n89YwF9LExnPIwgIVhoA': 210,
 '1M4n2XRaiIU9vON2R4SSRg': 165,
 '1p8Pu1pvAsuNtuZDtn2OkQ': 513,
 '26YpEJpa4uwJ60ezG9MIfQ': 164,
 '2XKlUjHGk9z-2ckFMcBEHw': 572,
 '4af7IWHRc890BUKcAGlyLQ': 194,
 '5Y5KbsI5buMcNh2hTFaRpA': 593,
 '5dYduxPNouuR5SI3V1OtCw': 191,
 '6toEhpmJxknHFKBw5u3xWQ': 172,
 '8LIfPfervWVYYpHWpjxTnQ': 197,
 '8pD-h427Y_aDcPxhDR7z8g': 202,
 '9TuUABNmDhBqdDael0zy8w': 234,
 '9XkpizE6dAqTmnyQ_eKLfA': 176,
 'AEowRtLGb_AdqFAZ8e6C9A': 284,
 'AVjnWX2F1VWEpkS0GMEGXQ': 246,
 'B0fzg0eUWplNBzfSJsCFDg': 247,
 'BVvWkghXzLYHWX-clzopdQ': 179,
 'B_RhT6CpAllq9NxtkpiDTA': 215,
 'CHYvZ-r1_bSMfBCswelaBw': 171,
 'CV4BMLxPSuC5V92JUfQlsw': 238,
 'DJBuyVr8_0k78ydUgonYdg': 190,
 'DbNKK25oOzfHxyfBHlOaDg': 180,
 'FVzA-5cUVRYttsv5aFOqQA': 309,
 'ISirqOjZ0HRaKJqbIuhGRQ': 223,
 'IcRIc6r4BA9dj_xUeNqLDA': 175,
 'Igs1V9jMkjdgNEhQBvE1bQ': 220,
 'JITevzyEgskIJc6sgOyEgw': 419,
 'Jc2D_Uk4i03E3bmFNp2Smw': 251,
 'JfqaznyuI0N4FnAXa-644Q': 215,
 'Jl4kC3eiGlEgEylXpOj7-Q': 238,
 'KGmDAZ

In [5]:
punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
corpus = []
for sent in review_df['text']:
    sent = sent.lower()
    words = word_tokenize(sent)
    words = [word for word in words if word not in punctuations]
    stopWords = set(stopwords.words('english'))
    words = [word for word in words if word not in stopWords]
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    corpus.append(' '.join(words))
    
cntVector = CountVectorizer(stop_words='english')
cntTf = cntVector.fit_transform(corpus)

lda = LatentDirichletAllocation(n_components=100, learning_offset=50, learning_method='online')
probs = lda.fit_transform(cntTf)
for i in range(100):
    name = 'topic_' + str(i+1)
    review_df[name] = probs[:,i]
review_df

,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,xXgE_rj-tK9v_MIH3mXpaQ,3W-lOOaIVQZONZP83kab6g,Reggie and company cook up the real deal at th...,5.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333,0.000333
1,8CSdj8LF593qwMcduzHglQ,3W-lOOaIVQZONZP83kab6g,Convential Wisdom is that this is the best res...,5.0,3.5,1.0,1.0,0.0,1.0,1.0,...,0.000233,0.000233,0.000233,0.121180,0.000233,0.068689,0.000233,0.000233,0.000233,0.000233
2,JjXEPAR_tf3VCdngVS9GMg,3W-lOOaIVQZONZP83kab6g,Finally an organic grocery store in East Nashv...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.000270,0.000270,0.000270,0.000270,0.000270,0.000270,0.000270,0.000270,0.037721,0.000270
3,cBHzDxKCsosdDI4UR3w-eg,3W-lOOaIVQZONZP83kab6g,"This converted laundromat is a small, trendy s...",3.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.000238,0.000238,0.000238,0.185725,0.000238,0.000238,0.000238,0.000238,0.000238,0.000238
4,tU692E8N0xBQ7Ogc78gN2g,3W-lOOaIVQZONZP83kab6g,This is a great place to meet all sorts of peo...,3.0,4.0,1.0,1.0,0.0,1.0,0.0,...,0.000145,0.000145,0.000145,0.000145,0.000145,0.000145,0.000145,0.000145,0.017045,0.000145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588250,wbaeORfUCcxkwdh7mNapRg,8j1OffLqKtetcAG3xv3f0Q,We had such a good time! Scott Hinds was very ...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.000588,0.000588,0.118235,0.075944,0.000588,0.000588,0.000588,0.000588,0.000588,0.091324
588251,cJGdlK7KeE78Qa0FydikIw,VTfl9PALOCiGV8SUBpCZEQ,We had a great buying experience. Michael Harr...,5.0,2.5,0.0,0.0,0.0,0.0,0.0,...,0.000588,0.000588,0.000588,0.000588,0.000588,0.000588,0.000588,0.059412,0.000588,0.000588
588252,U7esGceQIZ_1IG6Ib0c1lQ,m0-9woIKbK-R4F8RVEe4BQ,My colleague ordered on my behalf and she came...,5.0,2.5,1.0,1.0,0.0,1.0,0.0,...,0.000323,0.000323,0.032581,0.000323,0.000323,0.000323,0.000323,0.000323,0.000323,0.113362
588253,Y4uWSRQ-GyOaxA2c-qZxWw,QTVdM8ThpshrX-8GTcdk2w,Love this place! Great for toys and maternity...,5.0,3.5,0.0,0.0,0.0,0.0,0.0,...,0.000714,0.000714,0.000714,0.202565,0.000714,0.000714,0.000714,0.000714,0.000714,0.286284


In [6]:
# testing set
test_set = {}
user_groups = dict(list(review_df.groupby('user_id')))
business_groups = dict(list(review_df.groupby('business_id')))

In [8]:
user_profile = {}
for user, df in user_groups.items():
    p_hat = np.mean(np.array(df.iloc[:,14:]), axis=0)
    p = p_hat / np.sum(p_hat)
    user_profile[user] = p
business_profile = {}
for business, df in business_groups.items():
    q_hat = np.mean(np.array(df.iloc[:,14:]), axis=0)
    q = q_hat / np.sum(q_hat)
    business_profile[business] = q


for user in selected.keys():
    name = 'train_data/services.csv'
    test = pd.read_csv(name)
    del test['Unnamed: 0']
    del test['Ratings']
    probs = []
    for i in range(test.shape[0]):
        try:
            business = test.iloc[i, 0]
            theta = user_profile[user] * business_profile[business]
            theta = theta / np.sum(theta)
            probs.append(theta)
        except:
            theta = np.array([np.nan]*100)
            probs.append(theta)
    probs = np.array(probs)
    for i in range(100):
        name = 'topic_' + str(i+1)
        test[name] = probs[:,i]
    
    test = test.dropna(axis=0,how='any')
    test_set[user] = test

test_set.keys()

dict_keys(['-SigNCNItQDtSq27j0t_tw', '0-n89YwF9LExnPIwgIVhoA', '1M4n2XRaiIU9vON2R4SSRg', '1p8Pu1pvAsuNtuZDtn2OkQ', '26YpEJpa4uwJ60ezG9MIfQ', '2XKlUjHGk9z-2ckFMcBEHw', '4af7IWHRc890BUKcAGlyLQ', '5Y5KbsI5buMcNh2hTFaRpA', '5dYduxPNouuR5SI3V1OtCw', '6toEhpmJxknHFKBw5u3xWQ', '8LIfPfervWVYYpHWpjxTnQ', '8pD-h427Y_aDcPxhDR7z8g', '9TuUABNmDhBqdDael0zy8w', '9XkpizE6dAqTmnyQ_eKLfA', 'AEowRtLGb_AdqFAZ8e6C9A', 'AVjnWX2F1VWEpkS0GMEGXQ', 'B0fzg0eUWplNBzfSJsCFDg', 'BVvWkghXzLYHWX-clzopdQ', 'B_RhT6CpAllq9NxtkpiDTA', 'CHYvZ-r1_bSMfBCswelaBw', 'CV4BMLxPSuC5V92JUfQlsw', 'DJBuyVr8_0k78ydUgonYdg', 'DbNKK25oOzfHxyfBHlOaDg', 'FVzA-5cUVRYttsv5aFOqQA', 'ISirqOjZ0HRaKJqbIuhGRQ', 'IcRIc6r4BA9dj_xUeNqLDA', 'Igs1V9jMkjdgNEhQBvE1bQ', 'JITevzyEgskIJc6sgOyEgw', 'Jc2D_Uk4i03E3bmFNp2Smw', 'JfqaznyuI0N4FnAXa-644Q', 'Jl4kC3eiGlEgEylXpOj7-Q', 'KGmDAZI48MtoS_SnSkM0ag', 'KXDiP5LvmOIAH2M7rQyquw', 'KZV8cpBu4Ax1bZMucV3_Ww', 'Kc7HEHQUSO9NiEpb6C5fyg', 'Kho08pWNfeGnRbZTnMKy3w', 'L27OShvycGmVOkAxD1h86A', 'LMvGr9_gUrIwnD2iBS5MlQ', '

In [10]:
test_set['-SigNCNItQDtSq27j0t_tw']

,business_id,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,Smoking,Park,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,bBDDEgkFA1Otx9Lfe7BZUQ,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.000444,1.222307e-05,0.000234,0.000203,0.011680,4.158784e-04,0.004104,5.495113e-05,1.248507e-04,0.000013
1,tMkwHmWFUEXrC9ZduonpTg,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.000011,4.704802e-06,0.000006,0.000015,0.002365,1.513838e-05,0.002476,6.563285e-06,9.988139e-07,0.000011
2,lk9IwjZXqUMqqOhM774DtQ,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.000029,1.700898e-06,0.000011,0.000004,0.002419,1.617744e-05,0.000632,1.327067e-05,2.648800e-06,0.000015
3,B2qqjCl_BNNnXvWWfpFiLA,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000114,7.989669e-05,0.000731,0.000241,0.005144,1.162789e-04,0.000160,1.839611e-04,3.108620e-04,0.000101
4,oaboaRBUgGjbo2kfUIKDLQ,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000020,6.045906e-06,0.000003,0.000003,0.002227,3.077252e-05,0.000118,4.108463e-06,1.685401e-06,0.000007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8204,35JPTJ69zVoTGuk6TBX5qA,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.000051,5.699669e-06,0.000016,0.000007,0.003110,7.965102e-06,0.000093,5.057531e-06,1.490061e-05,0.000018
8205,Jv7_p4V9KG7Ar_GEXDenng,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000020,6.298965e-07,0.000020,0.000003,0.001573,8.472722e-07,0.000075,1.135497e-05,8.041117e-06,0.000006
8206,9iCum5W48TqNVDTZCJUKjg,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.000066,8.067819e-06,0.000005,0.000005,0.001797,6.624062e-06,0.000142,9.070356e-07,7.208430e-06,0.000034
8207,i6r6IdA-P8eGAgqZTUrQ3Q,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.000057,2.120682e-06,0.000004,0.000004,0.003512,8.471663e-06,0.000584,3.460290e-06,1.163954e-06,0.000008


In [33]:
def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5
    
# train 
train_x = review_df.iloc[:,5:]
train_y = review_df.iloc[:,4]
slr = LinearRegression()
slr.fit(train_x, train_y)

# test
pred_res = {}
for user, test_X in test_set.items():
    test_X = test_X.iloc[:,1:]
    y = slr.predict(test_X)
    y_pred = []
    for i in range(len(y)):
        tmp = (i, get_round_number(y[i]))
        y_pred.append(tmp)
    y_pred = sorted(y_pred, key=lambda x:x[1], reverse=True)
    pred_res[user] = y_pred
pred_res

{'-SigNCNItQDtSq27j0t_tw': [(1884, 4.5),
  (5919, 4.5),
  (31, 4.0),
  (51, 4.0),
  (56, 4.0),
  (81, 4.0),
  (144, 4.0),
  (227, 4.0),
  (230, 4.0),
  (265, 4.0),
  (300, 4.0),
  (311, 4.0),
  (335, 4.0),
  (337, 4.0),
  (350, 4.0),
  (378, 4.0),
  (388, 4.0),
  (391, 4.0),
  (452, 4.0),
  (463, 4.0),
  (497, 4.0),
  (503, 4.0),
  (525, 4.0),
  (557, 4.0),
  (583, 4.0),
  (584, 4.0),
  (612, 4.0),
  (643, 4.0),
  (661, 4.0),
  (666, 4.0),
  (696, 4.0),
  (759, 4.0),
  (783, 4.0),
  (816, 4.0),
  (820, 4.0),
  (856, 4.0),
  (868, 4.0),
  (890, 4.0),
  (912, 4.0),
  (915, 4.0),
  (927, 4.0),
  (932, 4.0),
  (951, 4.0),
  (969, 4.0),
  (995, 4.0),
  (1004, 4.0),
  (1024, 4.0),
  (1076, 4.0),
  (1077, 4.0),
  (1079, 4.0),
  (1110, 4.0),
  (1114, 4.0),
  (1149, 4.0),
  (1167, 4.0),
  (1169, 4.0),
  (1197, 4.0),
  (1200, 4.0),
  (1225, 4.0),
  (1277, 4.0),
  (1322, 4.0),
  (1324, 4.0),
  (1328, 4.0),
  (1374, 4.0),
  (1402, 4.0),
  (1404, 4.0),
  (1417, 4.0),
  (1460, 4.0),
  (1464, 4.0),
 

In [34]:
pred_res.keys()

dict_keys(['-SigNCNItQDtSq27j0t_tw', '0-n89YwF9LExnPIwgIVhoA', '1M4n2XRaiIU9vON2R4SSRg', '1p8Pu1pvAsuNtuZDtn2OkQ', '26YpEJpa4uwJ60ezG9MIfQ', '2XKlUjHGk9z-2ckFMcBEHw', '4af7IWHRc890BUKcAGlyLQ', '5Y5KbsI5buMcNh2hTFaRpA', '5dYduxPNouuR5SI3V1OtCw', '6toEhpmJxknHFKBw5u3xWQ', '8LIfPfervWVYYpHWpjxTnQ', '8pD-h427Y_aDcPxhDR7z8g', '9TuUABNmDhBqdDael0zy8w', '9XkpizE6dAqTmnyQ_eKLfA', 'AEowRtLGb_AdqFAZ8e6C9A', 'AVjnWX2F1VWEpkS0GMEGXQ', 'B0fzg0eUWplNBzfSJsCFDg', 'BVvWkghXzLYHWX-clzopdQ', 'B_RhT6CpAllq9NxtkpiDTA', 'CHYvZ-r1_bSMfBCswelaBw', 'CV4BMLxPSuC5V92JUfQlsw', 'DJBuyVr8_0k78ydUgonYdg', 'DbNKK25oOzfHxyfBHlOaDg', 'FVzA-5cUVRYttsv5aFOqQA', 'ISirqOjZ0HRaKJqbIuhGRQ', 'IcRIc6r4BA9dj_xUeNqLDA', 'Igs1V9jMkjdgNEhQBvE1bQ', 'JITevzyEgskIJc6sgOyEgw', 'Jc2D_Uk4i03E3bmFNp2Smw', 'JfqaznyuI0N4FnAXa-644Q', 'Jl4kC3eiGlEgEylXpOj7-Q', 'KGmDAZI48MtoS_SnSkM0ag', 'KXDiP5LvmOIAH2M7rQyquw', 'KZV8cpBu4Ax1bZMucV3_Ww', 'Kc7HEHQUSO9NiEpb6C5fyg', 'Kho08pWNfeGnRbZTnMKy3w', 'L27OShvycGmVOkAxD1h86A', 'LMvGr9_gUrIwnD2iBS5MlQ', '

In [35]:
pred_res['_wsoGbLalsSaZlfxUiWuCQ']

[(4345, 4.5),
 (5919, 4.5),
 (6831, 4.5),
 (6899, 4.5),
 (14, 4.0),
 (17, 4.0),
 (31, 4.0),
 (51, 4.0),
 (58, 4.0),
 (80, 4.0),
 (81, 4.0),
 (107, 4.0),
 (112, 4.0),
 (144, 4.0),
 (145, 4.0),
 (159, 4.0),
 (227, 4.0),
 (230, 4.0),
 (242, 4.0),
 (256, 4.0),
 (257, 4.0),
 (265, 4.0),
 (267, 4.0),
 (276, 4.0),
 (300, 4.0),
 (311, 4.0),
 (324, 4.0),
 (329, 4.0),
 (335, 4.0),
 (337, 4.0),
 (350, 4.0),
 (362, 4.0),
 (363, 4.0),
 (378, 4.0),
 (388, 4.0),
 (391, 4.0),
 (435, 4.0),
 (444, 4.0),
 (452, 4.0),
 (459, 4.0),
 (463, 4.0),
 (503, 4.0),
 (511, 4.0),
 (525, 4.0),
 (535, 4.0),
 (538, 4.0),
 (541, 4.0),
 (552, 4.0),
 (557, 4.0),
 (558, 4.0),
 (576, 4.0),
 (584, 4.0),
 (601, 4.0),
 (612, 4.0),
 (643, 4.0),
 (661, 4.0),
 (666, 4.0),
 (674, 4.0),
 (680, 4.0),
 (686, 4.0),
 (696, 4.0),
 (702, 4.0),
 (715, 4.0),
 (726, 4.0),
 (749, 4.0),
 (759, 4.0),
 (816, 4.0),
 (820, 4.0),
 (823, 4.0),
 (825, 4.0),
 (833, 4.0),
 (840, 4.0),
 (856, 4.0),
 (864, 4.0),
 (868, 4.0),
 (888, 4.0),
 (894, 4.0),
 (

In [51]:
all_top_30 = {}
for user in pred_res.keys():
    top_30 = {}
    for key, rating in pred_res[user]:
        if len(top_30) < 30:
            business = test_set[user].iloc[key, 0]
            top_30[business] = rating
        else:
            break
    all_top_30[user] = top_30
all_top_30

{'-SigNCNItQDtSq27j0t_tw': {'oZTDcsPKJfUGa0ksJyc1lQ': 4.5,
  'j2_7yAqPS4d1oMN5p5ydlQ': 4.5,
  'XNWVT7tR7yAzCyf0VQ-ijA': 4.0,
  'yTsG5uSr0OF0FuAuaa4MGA': 4.0,
  'WJkn1P09sKDpFrP2YHpJhw': 4.0,
  'x-HQh-lEouO2Dpn5_noaaw': 4.0,
  'mRAu0rgg_xxBtCdjj4bVdQ': 4.0,
  '9KLGDzlT-tk_vO_wRYSv0w': 4.0,
  'kingPUNT6SC8idmqMMrKjQ': 4.0,
  'eeoM7C1G099zj0huSqQfqQ': 4.0,
  'UNxpf7naQ1sGXGZ6zl81YQ': 4.0,
  'FzgjWbWARZBl-u622FpAmQ': 4.0,
  'IzPaKFuTapl94B6iMjz3sw': 4.0,
  '2lRNoZXy3BkN3XPrdC-vlQ': 4.0,
  'mzkitz3hHL9Xuk7Tk1AN7g': 4.0,
  'HPLMyOVlX3DbInKLZdLbtw': 4.0,
  'vpz_l8QIPSKRU8LGwfbb4A': 4.0,
  'aYgeoRkTFboe61Q6fJcv-w': 4.0,
  'efou6IrwQhHAXZ0e0v0Gkw': 4.0,
  'gns8jW8ntiM8hYoyl7gloA': 4.0,
  'dL2hANhOK-V9iF60UVkLAw': 4.0,
  'E8aqYBuJwFs7lp1YwEc6sg': 4.0,
  '-MnKKFnVjphA-yUzIMv0ew': 4.0,
  'g5gS1JZEp6q6j56bvZL9wQ': 4.0,
  'yoWl-47NCfhdOZ4YvLwVBA': 4.0,
  'js3C2umns9WtGBIBrQKzyQ': 4.0,
  '5csuCiUAgIgvL4D999huoQ': 4.0,
  'K3zSIfOUtYRrcch-bIS87w': 4.0,
  'Q5PLKRgua5U7D8C5ub5ZfQ': 4.0,
  'Dp_yOU1VrnWDtn

In [52]:
business_path = 'selected_data/business.json'
df_business = (pd.read_json(business_path)).T

In [65]:
for user in all_top_30.keys():
    folder1 = 'demo_res/' + user + '/history'
    folder2 = 'demo_res/' + user + '/predition'
    os.makedirs(folder1)
    os.makedirs(folder2)
    for i in range(user_groups[user].shape[0]):
        business = user_groups[user].iloc[i, 0]
        pd.DataFrame(df_business.loc[business]).to_csv('demo_res/' + user + '/history/' + business + '.csv')
    ct = 1
    for business in all_top_30[user].keys():
        pd.DataFrame(df_business.loc[business]).to_csv('demo_res/' + user + '/predition/' + str(ct) + '_' + business + '.csv')
        ct += 1